In [1]:
import json
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from functools import reduce
from IPython.display import HTML, display
from tabulate import tabulate

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables_original_singletons = get_markable_dataframe("data/testing/markables.csv", word_vector, idx_by_word)
markables_predicted_singletons = get_markable_dataframe("data/testing/markables_with_predicted_singleton.csv", word_vector, idx_by_word)

markables_original_singletons.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1916,"[1263, 1264, 1968, 1395]",0,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]",1,0,[],"[999, 379, 1161, 213, 27, 1263, 1969, 1188, 14...","[1.0, 0.0]"
1,1917,[213],1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",0,0,"[1263, 1264, 1968, 1395, 999, 379, 1161]","[27, 1263, 1969, 1188, 1470, 25, 1161, 63, 424...","[1.0, 0.0]"
2,1918,"[1263, 1969, 1188]",0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",1,0,"[1263, 1264, 1968, 1395, 999, 379, 1161, 213, 27]","[1470, 25, 1161, 63, 424, 1223, 25, 1415, 1161...","[1.0, 0.0]"
3,1919,"[1470, 25, 1161]",0,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]",0,0,"[1968, 1395, 999, 379, 1161, 213, 27, 1263, 19...","[63, 424, 1223, 25, 1415, 1161, 876, 344, 213,...","[0.0, 1.0]"
4,1920,[424],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1161, 213, 27, 1263, 1969, 1188, 1470, 25, 11...","[1223, 25, 1415, 1161, 876, 344, 213, 406, 122...","[0.0, 1.0]"


In [4]:
def get_markable_text(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].text.values[0]])

def get_markable_previous_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].previous_words.values[0]])

def get_markable_next_words(idx):
    return ' '.join([word_by_idx[x] for x in markables_original_singletons[markables_original_singletons['id'] == idx].next_words.values[0]])

def get_markable(idx):
    return f'{get_markable_previous_words(idx)} [{get_markable_text(idx).upper()}] {get_markable_next_words(idx)}'.strip()

In [5]:
right_predicted_markables = markables_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) == markables_original_singletons['is_singleton'].map(lambda x: x[0])]
wrong_predicted_markables = markables_predicted_singletons[markables_predicted_singletons['is_singleton'].map(lambda x: x[0]) != markables_original_singletons['is_singleton'].map(lambda x: x[0])]

# Analysis of Wrongly Predicted Markables

## Label: singleton, predicted: non-singleton

In [6]:
false_negatives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
false_negative_texts = [get_markable(idx) for idx in false_negatives['id']]

In [7]:
data_to_analyze =  false_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 23
proper name: 15
pronoun: 23
pronoun without "nya": 0
first person: 0


In [8]:
data_to_analyze, text_data_to_analyze = false_negatives, false_negative_texts

display(HTML(tabulate([
    [i, idx, text] 
    for i, idx, text 
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

1,1923,saham dengan harga rp per saham atau total [NYA] senilai rp miliar kepada exedy corporation yang berkedudukan di
2,1926,senilai rp miliar kepada exedy corporation yang berkedudukan di [JEPANG] kami telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik
3,1946,sisi direksi komisaris dan pemegang saham utama lagi pula nilai [NYA] tidak cukup material yakni kurang persen dari pendapatan dan
4,1966,tanggal berlaku dia menambahkan permohonan pembebasan bm harus diajukan kepada [DIRJEN BEA CUKAI DEPKEU] dengan melampirkan antara lain npwp ; surat izin usaha dari
5,2253,pada maret mencapai persen demikian dikemukakan deputi gubernur [BANK INDONESIA] bi hartadi a sarwono di jakarta rabu ia mengatakan persentase
6,2254,mencapai persen demikian dikemukakan deputi gubernur bank indonesia [BI] hartadi a sarwono di jakarta rabu ia mengatakan persentase tersebut
7,2268,hartadi dalam acara sosialisasi program peningkatan teknologi industri tpt di [BI] jakarta hartadi mengatakan bahwa industri tpt sebenarnya merupakan salah satu
8,2274,yang ditetapkan pemerintah untuk dikembangkan namun perkembangan peningkatan kapasitas produksi [NYA] menghadapi kendala besar sementara persaingan semakin meningkat data perbankan menunjukkan
9,2278,kendala besar sementara persaingan semakin meningkat data perbankan menunjukkan besar [NYA] penyaluran kredit pada industri tpt non - kulit pada maret
10,2290,keengganan perbankan untuk memberikan pembiayaan pada industri tersebut membuat perkembangan [NYA] sungguh sangat memprihatinkan upaya yang dilakukan pemerintah untuk memberikan subsidi
11,2308,ini acara sosialisasi dapat memberikan akses informasi kepada perbankan khusus [NYA] mengenai bidang-bidang usaha yang memiliki prospek yang baik dan potensial


## Label: non-singleton, predicted: singleton

In [9]:
false_positives = wrong_predicted_markables[wrong_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
false_positive_texts = [get_markable(idx) for idx in false_positives['id']]

In [10]:
data_to_analyze =  false_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 0
proper name: 23
pronoun: 2
pronoun without "nya": 2
first person: 0


In [11]:
data_to_analyze, text_data_to_analyze = false_positives, false_positive_texts

display(HTML(tabulate([
    [i, idx, text] 
    for i, idx, text 
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

1,1916,[PT ASTRA OTOPARTS TBK] menjual kepemilikan saham nya di pt exedy indonesia sebanyak
2,1918,pt astra otoparts tbk menjual kepemilikan saham nya di [PT EXEDY INDONESIA] sebanyak saham dengan harga rp per saham atau
3,1925,per saham atau total nya senilai rp miliar kepada [EXEDY CORPORATION] yang berkedudukan di jepang kami telah menandatangani perjanjian pengikatan untuk
4,1927,rp miliar kepada exedy corporation yang berkedudukan di jepang [KAMI] telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik pt
5,1930,kami telah menandatangani perjanjian pengikatan untuk penjualan seluruh saham milik [PT ASTRA OTOPARTS] di pt exedy indonesia kepada exedy corporation kata sekretaris perusahaan
6,1931,pengikatan untuk penjualan seluruh saham milik pt astra otoparts di [PT EXEDY INDONESIA] kepada exedy corporation kata sekretaris perusahaan astra otoparts kartina rahayu
7,1932,saham milik pt astra otoparts di pt exedy indonesia kepada [EXEDY CORPORATION] kata sekretaris perusahaan astra otoparts kartina rahayu di jakarta senin
8,2311,memiliki prospek yang baik dan potensial untuk dibiayai sementara itu [DIRJEND INDUSTRI LOGAM MESIN TEKSTIL DEPARTEMEN PERINDUSTRIAN ANSARI BUKHARI] mengatakan sebenarnya pembiayaan perbankan ke sektor tpt masih cukup baik
9,2333,mulyani mengungkapkan bahwa kontribusi pasar modal terhadap produk domestik bruto [PDB] pada saat ini mencapai persen dengan kapitalisasi rp
10,2343,forum ii di jakarta selasa mengatakan kontribusi pasar modal terhadap [PDB] ini menunjukkan trend naik dibanding pada akhir yang hanya
11,2347,dibanding pada akhir yang hanya sebesar persen dari [PDB] dengan kapitalisasi rp triliun peningkatan ini juga ditunjukkan dari


# Analysis of Rightly Predicted Markables

## Non-Singletons

In [12]:
true_negatives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 1]
true_negative_texts = [get_markable(idx) for idx in true_negatives['id']]

In [13]:
data_to_analyze =  true_negatives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 26
proper name: 40
pronoun: 37
pronoun without "nya": 11
first person: 0


In [14]:
data_to_analyze, text_data_to_analyze = true_negatives, true_negative_texts

display(HTML(tabulate([
    [i, idx, text] 
    for i, idx, text 
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

1,1917,pt astra otoparts tbk menjual kepemilikan saham [NYA] di pt exedy indonesia sebanyak saham dengan harga rp
2,1933,astra otoparts di pt exedy indonesia kepada exedy corporation kata [SEKRETARIS PERUSAHAAN ASTRA OTOPARTS] kartina rahayu di jakarta senin menurut dia penjualan saham tersebut
3,1934,exedy indonesia kepada exedy corporation kata sekretaris perusahaan astra otoparts [KARTINA RAHAYU] di jakarta senin menurut dia penjualan saham tersebut akan berlaku
4,1937,sekretaris perusahaan astra otoparts kartina rahayu di jakarta senin menurut [DIA] penjualan saham tersebut akan berlaku efektif apabila syarat-syarat sebagaimana termuat
5,1940,sebagaimana termuat dalam perjanjian pengikatan jual beli saham telah terpenuhi [DIA] mengatakan transaksi tersebut tidak mengandung unsur benturan kepentingan dilihat dari
6,1952,persen dari pendapatan dan persen dari ekuitas tambah [NYA]
7,1953,[MENKEU SRI MULYANI INDRAWATI] membebaskan tarif akhir bea masuk bm atas impor bahan baku
8,1964,hari setelah ditetapkan selama satu tahun sejak tanggal berlaku [DIA] menambahkan permohonan pembebasan bm harus diajukan kepada dirjen bea cukai
9,2255,mencapai persen demikian dikemukakan deputi gubernur bank indonesia bi [HARTADI A SARWONO] di jakarta rabu ia mengatakan persentase tersebut termasuk tinggi karena
10,2258,gubernur bank indonesia bi hartadi a sarwono di jakarta rabu [IA] mengatakan persentase tersebut termasuk tinggi karena npl untuk industri perbankan
11,2266,melahirkan persepsi bahwa industri tpt termasuk kategori sunset industri kata [HARTADI] dalam acara sosialisasi program peningkatan teknologi industri tpt di bi


## Singletons

In [15]:
true_positives = right_predicted_markables[right_predicted_markables['is_singleton'].map(lambda x: x[0]) == 0]
true_positive_texts = [get_markable(idx) for idx in true_positives['id']]

In [16]:
data_to_analyze =  true_positives

print(f'"nya": {len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'proper name: {len(data_to_analyze[data_to_analyze["is_proper_name"] == 1])}')
print(f'pronoun: {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1])}')   
print(f'pronoun without "nya": {len(data_to_analyze[data_to_analyze["is_pronoun"] == 1]) - len(list(filter(lambda x: get_markable_text(x) == "nya", data_to_analyze["id"])))}')
print(f'first person: {len(data_to_analyze[data_to_analyze["is_first_person"] == 1])}')

"nya": 0
proper name: 96
pronoun: 7
pronoun without "nya": 7
first person: 0


In [17]:
data_to_analyze, text_data_to_analyze = true_positives, true_positive_texts

display(HTML(tabulate([
    [i, idx, text] 
    for i, idx, text 
    in zip(range(1, len(data_to_analyze) + 1), data_to_analyze['id'], text_data_to_analyze)
], tablefmt='html')))

1,1919,otoparts tbk menjual kepemilikan saham nya di pt exedy indonesia [SEBANYAK SAHAM] dengan harga rp per saham atau total nya senilai
2,1920,saham nya di pt exedy indonesia sebanyak saham dengan [HARGA] rp per saham atau total nya senilai rp
3,1921,nya di pt exedy indonesia sebanyak saham dengan harga [RP ] per saham atau total nya senilai rp miliar kepada
4,1922,exedy indonesia sebanyak saham dengan harga rp per [SAHAM] atau total nya senilai rp miliar kepada exedy corporation
5,1924,saham dengan harga rp per saham atau total nya [SENILAI RP MILIAR] kepada exedy corporation yang berkedudukan di jepang kami telah menandatangani
6,1928,kepada exedy corporation yang berkedudukan di jepang kami telah menandatangani [PERJANJIAN PENGIKATAN] untuk penjualan seluruh saham milik pt astra otoparts di pt
7,1929,berkedudukan di jepang kami telah menandatangani perjanjian pengikatan untuk penjualan [SELURUH SAHAM] milik pt astra otoparts di pt exedy indonesia kepada exedy
8,1935,exedy corporation kata sekretaris perusahaan astra otoparts kartina rahayu di [JAKARTA] senin menurut dia penjualan saham tersebut akan berlaku efektif apabila
9,1936,corporation kata sekretaris perusahaan astra otoparts kartina rahayu di jakarta [SENIN] menurut dia penjualan saham tersebut akan berlaku efektif apabila syarat-syarat
10,1938,perusahaan astra otoparts kartina rahayu di jakarta senin menurut dia [PENJUALAN SAHAM TERSEBUT] akan berlaku efektif apabila syarat-syarat sebagaimana termuat dalam perjanjian pengikatan
11,1939,saham tersebut akan berlaku efektif apabila syarat-syarat sebagaimana termuat dalam [PERJANJIAN PENGIKATAN JUAL BELI SAHAM] telah terpenuhi dia mengatakan transaksi tersebut tidak mengandung unsur benturan
